# 라이브러리 임포트

In [15]:
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver

from tqdm import tqdm
from app_store_scraper import AppStore

import time
import pandas as pd
import numpy as np
import random


## PlayStore 리뷰 수집

## AppStore 리뷰 수집

In [16]:
def Extract_reviews(app_name:str, app_id:int): 
  pay = AppStore(country='kr', app_name=app_name, app_id=app_id)
  pay.review()

  pay_reviews = pd.DataFrame(pay.reviews).sort_values('date', ascending=False).reset_index(drop=True)
  pay_reviews['pay'] = app_name
  return pay_reviews

In [20]:
pay_app = {'kakaopay': 1464496236, 'naverpay': 1554807824, 'toss': 839333328}

app_store_reviews = pd.DataFrame()
for i in tqdm(pay_app):
  pay_reviews = Extract_reviews(i, pay_app[i])
  app_store_reviews= pd.concat([app_store_reviews, pay_reviews], axis=0)

print('Done..')

  0%|          | 0/3 [00:00<?, ?it/s]2025-01-26 15:59:37,955 [INFO] Base - Initialised: AppStore('kr', 'kakaopay', 1464496236)
2025-01-26 15:59:37,955 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/kakaopay/id1464496236
2025-01-26 15:59:43,620 [INFO] Base - [id:1464496236] Fetched 240 reviews (240 fetched in total)
2025-01-26 15:59:49,448 [INFO] Base - [id:1464496236] Fetched 480 reviews (480 fetched in total)
2025-01-26 15:59:55,398 [INFO] Base - [id:1464496236] Fetched 700 reviews (700 fetched in total)
2025-01-26 16:00:00,930 [INFO] Base - [id:1464496236] Fetched 900 reviews (900 fetched in total)
2025-01-26 16:00:06,409 [INFO] Base - [id:1464496236] Fetched 1060 reviews (1060 fetched in total)
2025-01-26 16:00:12,064 [INFO] Base - [id:1464496236] Fetched 1320 reviews (1320 fetched in total)
2025-01-26 16:00:17,504 [INFO] Base - [id:1464496236] Fetched 1544 reviews (1544 fetched in total)
 33%|███▎      | 1/3 [00:40<01:20, 40.15s/it]2025-01-26 16:00:19,506 

Done..


In [21]:
app_store_reviews

,date,developerResponse,review,rating,isEdited,userName,title,pay
0,2025-01-24 20:33:50,NaN,틱 없어진거 실화임? 업뎃 왜 한건지 모르겠고 진짜 구리다 다시 돌려놔라 주식 안하...,1,False,될놈될놈,일을 왜 이렇게함?,kakaopay
1,2025-01-24 13:10:23,NaN,정보 왜 팔아?,1,False,소녀의세계최고,카톡 뭐하니?,kakaopay
2,2025-01-24 09:07:11,NaN,4천만명 개인정보 넘긴거 짘짜냐고,1,False,옹ㅇ옹ㅇ옹,고객 개인정보 중국에 넘긴거 진짠가요,kakaopay
3,2025-01-22 08:50:54,"{'id': 39865764, 'body': '안녕하세요, 도후경수님. 카카오페이 ...","아, 업데이트 전으로 해주세요~\n거래대금, 기관, 외국인수급 다없어졌어요 보는 지...",3,True,도후경수,카카오증권,kakaopay
4,2025-01-22 04:01:10,NaN,대체 왜 업데이트 한거죠??? 주식하려고 어플쓰는데 차트 왜 이러죠?? 틱이 없는게...,1,False,걸음수,최악,kakaopay
...,...,...,...,...,...,...,...,...
635,2017-11-01 13:50:10,NaN,처음에 여기저기 광고 하는거 봤을때는 뭔가 거짓말일 것 같고 이것저것 복잡할 것 같...,5,False,새로운별명이길,확실히 편하네요,toss
636,2017-10-27 02:29:04,NaN,너무 잘쓰고 있습니당 ㅎㅎ 친구들한테도 추천해주고 다녔어요 너~~~무 편해요 월 5...,5,False,zumdol,토스,toss
637,2017-10-26 12:58:30,NaN,몇년째 엄청 잘쓰고 있어요!ㅎㅎ 자동으로 한달에 얼마씩 빠져나가게 설정해서 조그만 ...,5,False,탄빵쌤,리뷰남겨본적 한번도 없지만 토스라면!!,toss
638,2017-10-25 17:46:58,NaN,한창 토스로 송금하면 귀찮다고 이게 뭐냐 하던 때부터 사용했던 사람입니다. 지금은 ...,5,False,냔리,초창기부터 썼던 사용자입니다,toss


# 파일 위치 설정

In [ ]:
# 크롬 드라이버 제어
driver = webdriver.Chrome()